In [51]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.utils.colab.set_colab_file_id('1v0PiRlSXYTrNrHQcuHEQ41drvAptp_ug')

# Using data to shop the best lingerie

In this notebook I will be doing an exploratory analysis on  June 2017-July 2017 lingerie data from popular retailers such as: Amazon, American Eagle, b.tempt'd, Calvin Klein, Hanky Panky , Macy's, Nordstrom and Topshop USA.


The focus on this notebook will be:
- Analysis on the general quality of the brand measured by the ratings of their products.
- Analysis of quality, price and discounts in other to find the lingerie with a good relationship quality price.
- Analysis of the difference in price depending on the brand and retailer.
- Finding common style attributes.
- Analysis of the sizes available





__NOTE:__ This project was done as part of the [Jovian.ml](https://jovian.ml) Zero to Pandas [course](http://zerotopandas.com). This notebook was created to run on a colab instance and use a Kaggle API key to access the data.

__Dataset:__

[Innerwear Data from Victoria's Secret and Others](https://www.kaggle.com/PromptCloudHQ/innerwear-data-from-victorias-secret-and-others)

__Inspiration:__


[EDA for Lingerie](https://www.kaggle.com/justinekays/eda-for-lingerie)

[Data Driven Lingerie Shopping](https://www.kaggle.com/jkokatjuhha/data-driven-lingerie-shopping)

Questions to answer:



In [48]:
project_name = "zerotopandas-course-project" # change this

In [ ]:
!pip install jovian --upgrade -q

In [ ]:
import jovian

In [ ]:
jovian.commit(project=project_name)

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
[jovian] Committed successfully! https://jovian.ml/mvortizr/zerotopandas-course-project


'https://jovian.ml/mvortizr/zerotopandas-course-project'

### Getting the data


In [ ]:
# Colab library to upload files to notebook
from google.colab import files

In [ ]:
# Install Kaggle library
!pip install -q kaggle

__Connecting with Kaggle__


In [ ]:
# Upload kaggle API key file
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!touch ~/.kaggle/kaggle.json

In [ ]:
!cp kaggle.json ~/.kaggle/kaggle.json

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
#Download datasets from Kaggle 
!kaggle datasets download -d PromptCloudHQ/innerwear-data-from-victorias-secret-and-others

innerwear-data-from-victorias-secret-and-others.zip: Skipping, found more recently modified local copy (use --force to force download)


__Loading the data__

In [ ]:
!sudo apt-get install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [ ]:
!unzip innerwear-data-from-victorias-secret-and-others.zip -d datasets

Archive:  innerwear-data-from-victorias-secret-and-others.zip
  inflating: datasets/ae_com.csv     
  inflating: datasets/amazon_com.csv  
  inflating: datasets/btemptd_com.csv  
  inflating: datasets/calvinklein_com.csv  
  inflating: datasets/hankypanky_com.csv  
  inflating: datasets/macys_com.csv  
  inflating: datasets/shop_nordstrom_com.csv  
  inflating: datasets/us_topshop_com.csv  
  inflating: datasets/victoriassecret_com.csv  


## __Data Preparation and Cleaning__


In [70]:
import pandas as pd

### __Creating the initial dataframes__

I will load all the `.csv` files on different dataframes, each dataframe belongs to a different retailer.


In [144]:
american_eagle_df = pd.read_csv('./datasets/ae_com.csv')
amazon_df = pd.read_csv('./datasets/amazon_com.csv')
btemptd_df = pd.read_csv('./datasets/btemptd_com.csv')
calvin_klein_df = pd.read_csv('./datasets/calvinklein_com.csv')
hanky_panky_df = pd.read_csv('./datasets/hankypanky_com.csv')
macys_df = pd.read_csv('./datasets/macys_com.csv')
nordstrom_df = pd.read_csv('./datasets/shop_nordstrom_com.csv')
topshop_df = pd.read_csv('./datasets/us_topshop_com.csv')
victorias_secret_df = pd.read_csv('./datasets/victoriassecret_com.csv')


### __Exploring the dataframes__

All the dataframes have the same columns:
- `product_name`: name of the product
- `mrp`: maximun retail price
- `price`: price of the product
- `pdp_url`: url of the product
- `brand_name`: name of the brand
- `product_category`: category of the product
- `retailer`: retailer of the product
- `description`: description of the product
- `rating`: rating of the product
- `review count`: number of reviews the product received
- `style_attributes`: styles attributes of the product
- `total_sizes`: total sizes of the product
- `available_sizes`: available sizes of the product
- `color`: colors available of the product

In [145]:
american_eagle_df.columns

Index(['product_name', 'mrp', 'price', 'pdp_url', 'brand_name',
       'product_category', 'retailer', 'description', 'rating', 'review_count',
       'style_attributes', 'total_sizes', 'available_size', 'color'],
      dtype='object')

### __Cleaning the brand names__

If we see the unique values of the column `brand_name` of each retailer, we can notice several inconsistencies:

1. Existence of links instead of strings in the amazon dataframe

2. Each retailer typed the name of the brands differently. For example, Macy's wrote `Wacoal` but nordstrom wrote `WACOAL` or `B.TEMPT'D BY WACOAL`. 

We want to keep the name of every brand consistent across the different dataframes.

We will keep the dataframes from American Eagle, Victoria's Secret and Calvin Klein as they are.

In [146]:
american_eagle_df.brand_name.unique(), calvin_klein_df.brand_name.unique(),victorias_secret_df.brand_name.unique()

(array(['AERIE', 'AEO'], dtype=object),
 array(['Calvin Klein'], dtype=object),
 array(["Victoria's Secret", "Victoria's Secret Pink"], dtype=object))

__Cleaning the amazon dataframe brand names__

In [147]:
amazon_df.brand_name.unique()

array(['Calvin-Klein', 'Wacoal', 'Hanky-Panky', 'b-temptd',
       'Victorias-Secret', 's',
       'ref=w_bl_sl_l_ap_ap_web_2586685011?ie=UTF8&node=2586685011&field-lbr_brands_browse-bin=Calvin+Klein',
       'ref=w_bl_sl_l_b_ap_web_2586451011?ie=UTF8&node=2586451011&field-lbr_brands_browse-bin=b.tempt%27d',
       'Fila', 'Lucky-Brand',
       'ref=w_bl_sl_l_b_ap_web_2603426011?ie=UTF8&node=2603426011&field-lbr_brands_browse-bin=Wacoal',
       'Creative-Motion', 'Sexy-Hair', 'Nintendo', 'Compression-Comfort'],
      dtype=object)

Some brands listed look like wrong data. For instance, it seems suspicious that Nintendo is listed as a brand, I don't think Nintendo does Lingerie... let's inspect that.

In [148]:
amazon_df[amazon_df.brand_name == 'Nintendo']

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color
29424,b.tempt'd by Wacoal Women's Ciao Bella Balcone...,$38.00,$30.40,https://www.amazon.com/-/dp/B015O76DM2?th=1&psc=1,Nintendo,Bras,Amazon US,Balconette features corded lace overlay with a...,4.3,375,"[ 86% Nylon/14% Spandex , Imported , Hook and ...","30C , 30D , 30DD , 30DDD , 32B , 32C , 32D , 3...","30C , 30D , 30DD , 30DDD , 32B , 32C , 32D , 3...",Peacoat


In fact, it appears to be a btempt wacoal product that was listed as "Nintendo", it is better to drop this product

In [149]:
amazon_df.drop(amazon_df[amazon_df.brand_name == 'Nintendo'].index, inplace=True)

Now, let's watch other suspicious brands such as 'Creative-Motion', 'Sexy-Hair', 's','Lucky-Brand' and 'Compression-Comfort'

In [102]:
amazon_df[amazon_df.brand_name == 'Creative-Motion']

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color
21008,b.tempt'd by Wacoal Women's Ciao Bella Balcone...,$38.00,$21.89,https://www.amazon.com/-/dp/B00V9RV5M0?th=1&psc=1,Creative-Motion,Bras,Amazon US,Balconette features corded lace overlay with a...,4.3,376,"[ 86% Nylon/14% Spandex , Imported , Hand Wash...","30C , 30D , 30DD , 30DDD , 32B , 32C , 32D , 3...","30DD , 32B , 32C , 32D , 32DD , 34C , 34D , 36...",Dazzling Blue


In [103]:
amazon_df[amazon_df.brand_name == 's']

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color
935,Wacoal Women's Front Close T-Back Bra,$46.00,$46.00,https://www.amazon.com/-/dp/B0002NXT4Y?th=1&psc=1,s,Bras,Amazon US,Pure comfort in a seamless molded front close ...,4.2,742,"[ Fabric: 87% Nylon , 13% Spandex; Top cup: 10...","32C , 32D , 32DD , 34B , 34C , 34D , 34DD , 36...","32C , 32D , 32DD , 34B , 34C , 34D , 36B , 36C...",Black
10417,b.tempt'd by Wacoal Women's Ciao Bella Balcone...,$38.00,$24.95,https://www.amazon.com/-/dp/B01FXTD1XK?th=1&psc=1,s,Bras,Amazon US,Balconette features corded lace overlay with a...,4.3,377,"[ 86% Nylon/14% Spandex , Imported , Hand Wash...","30C , 30D , 30DD , 30DDD , 32B , 32C , 32D , 3...","30C , 30D , 30DD , 30DDD , 32B , 32C , 32D , 3...",Baja Blue
28997,Wacoal Women's Embrace Lace Hi-Cut Brief Panty,$27.00,$27.00,https://www.amazon.com/-/dp/B006M0J8Q4?th=1&psc=1,s,Panties,Amazon US,Embrace lace hi-cut brief offers great fit and...,4.5,23,"[ Body: 82% Nylon, 18% Spandex , Imported , Ha...","Small , Medium , Large , X-Large , 2X , 2X Plus","Medium , Large , X-Large , 2X",Small


In [104]:
amazon_df[amazon_df.brand_name == 'Compression-Comfort']

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color
29805,Wacoal Women's Ultimate Side Contour Bra,$68.00,$68.00,https://www.amazon.com/-/dp/B013V1521G?th=1&psc=1,Compression-Comfort,Bras,Amazon US,Ultimate side smoother contour t-shirt bra off...,3.7,57,"[ 58% Nylon, 42% Spandex , Imported , Hook and...","30C , 30D , 30DD , 30DDD , 30G , 32C , 32D , 3...","30C , 30D , 30DD , 30DDD , 30G , 32C , 32D , 3...",Black


In [106]:
amazon_df[amazon_df.brand_name == 'Sexy-Hair']

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color
26158,Wacoal Women's Full Figure Basic Beauty Underw...,$54.87,$54.87,https://www.amazon.com/-/dp/B008DKBZ9E?th=1&psc=1,Sexy-Hair,Bras,Amazon US,Seamless molded two-ply cups with inner sling ...,4.2,959,"[ 85% Nylon/15% Spandex , Imported , Hook and ...","32D , 32DD , 32DDD , 32G , 34C , 34D , 34DD , ...","32D , 32DD , 32DDD , 34C , 34D , 34DD , 34DDD ...",Cappuccino


In [107]:
amazon_df[amazon_df.brand_name == 'Lucky-Brand']

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color
8906,Wacoal Women's Embrace Lace Bikini Panty,$15.07,$15.07,https://www.amazon.com/-/dp/B0147GNOP0?th=1&psc=1,Lucky-Brand,Panties,Amazon US,Embrace lace bikini offers great fit and match...,4.3,94,"[ 100% Nylon , Imported , Hand Wash , Scallope...","Small , Medium , Large , X-Large","Small , X-Large",Large


It appears that all of them are Btemptd products that were mislabelled as well. Let's drop them all

In [150]:
amazon_df.drop(amazon_df[amazon_df.brand_name == 'Creative-Motion'].index, inplace=True)
amazon_df.drop(amazon_df[amazon_df.brand_name == 's'].index, inplace=True)
amazon_df.drop(amazon_df[amazon_df.brand_name == 'Compression-Comfort'].index, inplace=True)
amazon_df.drop(amazon_df[amazon_df.brand_name == 'Sexy-Hair'].index, inplace=True)
amazon_df.drop(amazon_df[amazon_df.brand_name == 'Lucky-Brand'].index, inplace=True)

Now we want to standarize the name of the brands left

In [151]:
amazon_df.brand_name.unique()

array(['Calvin-Klein', 'Wacoal', 'Hanky-Panky', 'b-temptd',
       'Victorias-Secret',
       'ref=w_bl_sl_l_ap_ap_web_2586685011?ie=UTF8&node=2586685011&field-lbr_brands_browse-bin=Calvin+Klein',
       'ref=w_bl_sl_l_b_ap_web_2586451011?ie=UTF8&node=2586451011&field-lbr_brands_browse-bin=b.tempt%27d',
       'Fila',
       'ref=w_bl_sl_l_b_ap_web_2603426011?ie=UTF8&node=2603426011&field-lbr_brands_browse-bin=Wacoal'],
      dtype=object)

In [152]:
def standarize_amazon_brand_names(brand_name):
  if brand_name in ['Calvin-Klein','ref=w_bl_sl_l_ap_ap_web_2586685011?ie=UTF8&node=2586685011&field-lbr_brands_browse-bin=Calvin+Klein']:
    brand_name = 'Calvin Klein'
  elif brand_name in ['ref=w_bl_sl_l_b_ap_web_2603426011?ie=UTF8&node=2603426011&field-lbr_brands_browse-bin=Wacoal','Wacoal', 'b-temptd','ref=w_bl_sl_l_b_ap_web_2586451011?ie=UTF8&node=2586451011&field-lbr_brands_browse-bin=b.tempt%27d']:
    brand_name = "b.tempt'd"
  elif brand_name == 'Hanky-Panky':
    brand_name = 'Hanky Panky'
  elif brand_name == 'Victorias-Secret':
    brand_name = "Victoria's Secret"
  return brand_name

In [153]:
amazon_df['brand_name'] = amazon_df['brand_name'].apply(standarize_amazon_brand_names)

And it's done!

In [154]:
amazon_df.brand_name.unique()

array(['Calvin Klein', "b.tempt'd", 'Hanky Panky', "Victoria's Secret",
       'Fila'], dtype=object)

__Cleaning b.tempt'd brand name__

In [155]:
btemptd_df.brand_name.unique()

array(['WACOAL'], dtype=object)

In [156]:
btemptd_df.brand_name = "b.tempt'd"

In [157]:
btemptd_df.brand_name.unique()

array(["b.tempt'd"], dtype=object)

__Cleaning the Hanky Panky brand name__

In [158]:
hanky_panky_df.brand_name.unique()

array(['HankyPanky'], dtype=object)

In [159]:
hanky_panky_df.brand_name = 'Hanky Panky'

In [160]:
hanky_panky_df.brand_name.unique()

array(['Hanky Panky'], dtype=object)

__Cleaning Macy's brand names__

In [161]:
macys_df.brand_name.unique()

array(['Calvin Klein', 'Wacoal', 'Hanky Panky', "b.tempt'd by Wacoal",
       'Calvin Klein Modern Cotton', 'Vanity Fair',
       'Calvin Klein Performance', "b.tempt'd"], dtype=object)

In [162]:
def standarize_macys_brand_names(brand_name):
  if brand_name in ['Calvin Klein','Calvin Klein Modern Cotton','Calvin Klein Performance']:
    brand_name = 'Calvin Klein'
  elif brand_name in ['Wacoal',"b.tempt'd by Wacoal"]:
    brand_name = "b.tempt'd"
  return brand_name

In [163]:
macys_df['brand_name'] = macys_df['brand_name'].apply(standarize_macys_brand_names)

In [164]:
macys_df.brand_name.unique()

array(['Calvin Klein', "b.tempt'd", 'Hanky Panky', 'Vanity Fair'],
      dtype=object)

__Clean nordstrom dataframe__

In [165]:
nordstrom_df.brand_name.unique()

array(['WACOAL', 'HANKY PANKY', "B.TEMPT'D BY WACOAL", 'CALVIN KLEIN',
       'NORDSTROM LINGERIE'], dtype=object)

In [169]:
def standarize_nordstrom_brand_names(brand_name):
  if brand_name == 'CALVIN KLEIN':
    brand_name = 'Calvin Klein'
  elif brand_name in ['WACOAL',"B.TEMPT'D BY WACOAL"]:
    brand_name = "b.tempt'd"
  elif brand_name == 'HANKY PANKY':
    brand_name = 'Hanky Panky'
  elif brand_name == 'NORDSTROM LINGERIE':
    brand_name = 'Nordstrom'
  return brand_name

In [170]:
nordstrom_df['brand_name'] = nordstrom_df['brand_name'].apply(standarize_nordstrom_brand_names)

In [171]:
nordstrom_df.brand_name.unique()

array(["b.tempt'd", 'Hanky Panky', 'Calvin Klein', 'Nordstrom'],
      dtype=object)

__Clean Top Shop brand name__

In [166]:
topshop_df.brand_name.unique()

array(['US TOPSHOP'], dtype=object)

In [167]:
topshop_df.brand_name = 'Top Shop'

In [168]:
topshop_df.brand_name.unique()

array(['Top Shop'], dtype=object)

### __Cleaning the retailers names__

The names displayed on the retailer column must be the correct spelling of the brand. We know every retailer is from the US so let's remove that information

In [181]:
american_eagle_df.retailer = 'American Eagle'
amazon_df.retailer = 'Amazon'
btemptd_df.retailer = "b.tempt'd"
calvin_klein_df.retailer = 'Calvin Klein'
hanky_panky_df.retailer = 'Hanky Panky'
macys_df.retailer = "Macy's"
nordstrom_df.retailer = 'Nordstrom'
topshop_df.retailer = 'Top Shop'
victorias_secret_df.retailer = "Victoria's Secret"

### __Merging all dataframes in one__

In [182]:
lingerie_df = american_eagle_df
lingerie_df.append(amazon_df)
lingerie_df.append(btemptd_df)
lingerie_df.append(calvin_klein_df)
lingerie_df.append(hanky_panky_df)
lingerie_df.append(macys_df)
lingerie_df.append(nordstrom_df)
lingerie_df.append(topshop_df)
lingerie_df.append(victorias_secret_df)

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color
0,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,https://www.ae.com/aerie-everyday-loves-lace-c...,AERIE,Cheekies,American Eagle,Introducing Everyday Loves™: Made with love. E...,5.0,8.0,"[""Soft lace with the right amount of stretch"",...","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Rugged Green
1,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,https://www.ae.com/aerie-everyday-loves-lace-c...,AERIE,Cheekies,American Eagle,Introducing Everyday Loves™: Made with love. E...,5.0,8.0,"[""Soft lace with the right amount of stretch"",...","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Natural Nude
2,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,https://www.ae.com/aerie-everyday-loves-lace-c...,AERIE,Cheekies,American Eagle,Introducing Everyday Loves™: Made with love. E...,5.0,8.0,"[""Soft lace with the right amount of stretch"",...","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",True Black
3,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,https://www.ae.com/aerie-everyday-loves-lace-c...,AERIE,Cheekies,American Eagle,Introducing Everyday Loves™: Made with love. E...,5.0,8.0,"[""Soft lace with the right amount of stretch"",...","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",White
4,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,https://www.ae.com/aerie-everyday-loves-lace-c...,AERIE,Cheekies,American Eagle,Introducing Everyday Loves™: Made with love. E...,5.0,8.0,"[""Soft lace with the right amount of stretch"",...","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Royal Navy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453381,Dream Angels NEW! Lace Mini Slip,$48.00,$48.00,https://www.victoriassecret.com/lingerie/shop-...,Victoria's Secret,Lace Mini Slip,Victoria's Secret,"All you need is a little lace, here in the cut...",NaN,NaN,NaN,"[""XS"", ""S"", ""M"", ""L"", ""XL""]",L,coconut white
453382,Dream Angels NEW! Lace Mini Slip,$48.00,$48.00,https://www.victoriassecret.com/lingerie/shop-...,Victoria's Secret,Lace Mini Slip,Victoria's Secret,"All you need is a little lace, here in the cut...",NaN,NaN,NaN,"[""XS"", ""S"", ""M"", ""L"", ""XL""]",L,mulled grape
453383,Dream Angels NEW! Lace Mini Slip,$48.00,$48.00,https://www.victoriassecret.com/lingerie/shop-...,Victoria's Secret,Lace Mini Slip,Victoria's Secret,"All you need is a little lace, here in the cut...",NaN,NaN,NaN,"[""XS"", ""S"", ""M"", ""L"", ""XL""]",M,sheer pink
453384,Dream Angels NEW! Lace Mini Slip,$48.00,$48.00,https://www.victoriassecret.com/lingerie/shop-...,Victoria's Secret,Lace Mini Slip,Victoria's Secret,"All you need is a little lace, here in the cut...",NaN,NaN,NaN,"[""XS"", ""S"", ""M"", ""L"", ""XL""]",M,coconut white


## Exploratory Analysis and Visualization


## Asking and Answering Questions



## Inferences and Conclusion

TODO

In [ ]:
import jovian

In [ ]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "aakashns/zerotopandas-course-project-starter" on https://jovian.ml/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ml/aakashns/zerotopandas-course-project-starter


'https://jovian.ml/aakashns/zerotopandas-course-project-starter'

## References and Future Work

TODO

In [ ]:
import jovian

In [ ]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
